In [1]:
import sys
import localSettings as ls
print(ls.main_path)

/home/david/data/searchingfornues/v08_00_00_33/cc0pinp/0109/


In [2]:
main_path = ls.main_path
sys.path.append(main_path)

In [3]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

date and time: 01282020


In [4]:
SHRCALIBFACTOR = 1.00

In [5]:
# USING BDT?
USEBDT = False

In [6]:
# Doing Run3?
ISRUN3 = False

In [7]:
# scale to MCC8 CV?
MCC8WEIGHTS = False

In [8]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [9]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

FnumuCV = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2.root"
numu_cv  = uproot.open(ls.main_path+FnumuCV)[fold][tree]

variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category", "shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "crtveto","crthitpe",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP",
    "weightSpline","weightTune","weightSplineTimesTune",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    #"pi0_energy2_Y", # pi0 tagger variables
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2"
]

#make the list unique
variables = list(set(variables))
print(variables)

FnuCV = "prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2"
nu_cv  = uproot.open(ls.main_path+FnuCV+".root")[fold][tree]

nuup_var_v = []
nu_var_fname_v = ["prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodangle_mcc9.1_v08_00_00_30_run1_reco2_reco2",\
                  "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodx_mcc9.1_v08_00_00_30_run1_reco2_reco2",\
                  "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodyz_mcc9.1_v08_00_00_31_run1_reco2_reco2"]
    
for fname in nu_var_fname_v:

    nu_var = uproot.open(ls.main_path+"/detsys/"+fname+".root")[fold][tree]
    nuup_var_v.append(nu_var)


['mc_pdg', 'trk_chipr', 'run', 'shrclusfrac1', 'category', 'weightSpline', 'trk_score_v', 'reco_nu_vtx_sce_y', 'trk_energy_tot', 'shr_dedx_U', 'slnunhits', 'shr_tkfit_dedx_V', 'reco_nu_vtx_sce_z', 'shr_tkfit_gap10_dedx_U', 'shr_energy', 'shr_tkfit_dedx_U', 'trk_hits_tot', 'best_obviouscosmic_flashmatch_score', 'sub', 'trk_len', 'reco_nu_vtx_x', 'shr_tkfit_2cm_dedx_U', 'crthitpe', 'trk_score', 'shrclusfrac2', 'reco_nu_vtx_z', 'best_cosmic_flashmatch_score', 'shr_bkt_E', 'shr_theta', 'shr_tkfit_2cm_dedx_V', 'pion_e', 'trk_phi', 'shr_tkfit_npointsvalid', 'backtracked_pdg', 'hits_y', 'nproton', 'shr_hits_tot', 'NeutrinoEnergy2', 'pi0truth_elec_etot', 'trk_bkt_E', 'trk_pid_chipr_v', 'shr_tkfit_gap10_dedx_Y', 'shrsubclusters0', 'shr_distance', 'pt', 'selected', 'shr_energy_tot_cali', 'shr_bkt_pdg', 'secondshower_Y_vtxdist', 'shr_score', 'trkshrhitdist2', 'trk_theta', 'shr_dedx_Y', 'evnunhits', 'pi0_e', 'true_e_visible', 'hits_ratio', 'shr_tkfit_gap10_dedx_V', 'crtveto', 'slclustfrac', 'n_sho

In [10]:
uproot_v = [nu_cv]

for up in nuup_var_v:
    uproot_v.append(up)

nu_cv  = nu_cv.pandas.df(variables, flatten=False)

nu_var_v = []

for up in nuup_var_v:
    nu_var_v.append(up.pandas.df(variables, flatten=False))

In [11]:
df_v = [nu_cv]
for df in nu_var_v:
    df_v.append(df)

In [12]:
for i,df in enumerate(df_v):
    
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
    #df['weightSpline']  = df['weightSpline']  * df['weightTune']

In [13]:
# how to get the LLR-PID value for the "track candidate" (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!

for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])

In [14]:
# define some energy-related variables
for i,df in enumerate(df_v):
    df["reco_e"] = (df["shr_energy_tot_cali"] + 0.030) / 0.79 + df["trk_energy_tot"]


In [15]:
# variables to be trained on
TRAINVAR = [#"n_showers_contained",
            "shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_gap10_dedx_Y","shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V",
            "shr_tkfit_2cm_dedx_Y","shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V",
            "trkfit","trkpid",
            "subcluster","shrmoliereavg",#"shrmoliererms",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            #'secondshower_V_nhit','secondshower_V_vtxdist','secondshower_V_dot','anglediff_V',
            #'secondshower_U_nhit','secondshower_U_vtxdist','secondshower_U_dot','anglediff_U',
            #"pi0_radlen1","pi0_radlen2","pi0_dedx2_fit_Y",
            #"pi0_energy2_Y","pi0_mass_Y","pi0_gammadot",
           ]

In [16]:
LABELS = ['pi0','nonpi0']

if (USEBDT == True):
    #for label, bkg_query in zip(nue_booster.labels, nue_booster.bkg_queries):
    for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_dec.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            mc[label+"_score"] = booster.predict(
                xgb.DMatrix(mc[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            nue[label+"_score"] = booster.predict(
                xgb.DMatrix(nue[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ext[label+"_score"] = booster.predict(
                xgb.DMatrix(ext[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            data[label+"_score"] = booster.predict(
                xgb.DMatrix(data[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            dirt[label+"_score"] = booster.predict(
                xgb.DMatrix(dirt[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            lee[label+"_score"] = booster.predict(
                xgb.DMatrix(lee[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ncpi0[label+"_score"] = booster.predict(
                xgb.DMatrix(ncpi0[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ccpi0[label+"_score"] = booster.predict(
                xgb.DMatrix(ccpi0[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ccnopi[label+"_score"] = booster.predict(
                xgb.DMatrix(ccnopi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            nccpi[label+"_score"] = booster.predict(
                xgb.DMatrix(nccpi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ncnopi[label+"_score"] = booster.predict(
                xgb.DMatrix(ncnopi[TRAINVAR]),
                ntree_limit=booster.best_iteration)


In [ ]:
# 1eNp BDT
QUERY = 'nslice == 1'
QUERY += ' and selected == 1'
QUERY += ' and n_tracks_contained > 0'
QUERY += ' and shr_energy_tot_cali > 0.07'
#if ISRUN3: QUERY += ' and (crtveto!=1) and (_closestNuCosmicDist > 20.)'
    
# Loose box cuts
QUERY += ' and n_showers_contained == 1'
QUERY += ' and hits_ratio > 0.5'
QUERY += ' and tksh_distance < 6.0'
QUERY += ' and shr_tkfit_2cm_dedx_Y < 4.0'
QUERY += ' and tksh_angle > -0.9'
QUERY += ' and trkpid < 0.1'
QUERY += ' and shr_score < 0.30'
QUERY += ' and CosmicIP > 20.'  

# in case you do not want to look at the data
QUERY += ' and bnbdata==0'
    
# and BDT    
QUERY += ' and pi0_score > 0.995 and nonpi0_score > 0.9984'

In [25]:
# create output files based on selection

fname_v = [FnuCV]
for nu_var_fname in nu_var_fname_v:
    fname_v.append(nu_var_fname)

for i,name in enumerate(fname_v):
        
    df = df_v[i]

    fout = open(ls.ntuple_path+"txt/detsys/"+name+'.txt','w')
    dfsel = df.query(QUERY)
    print ('file %s has %i selected entries'%(name,dfsel.shape[0]))
    for i,row in dfsel.iterrows():
        run = row['run']
        sub = row['sub']
        evt = row['evt']
        fout.write('%i %i %i \n'%(run,sub,evt))
    fout.close()

file prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2 has 1880 selected entries
file prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodangle_mcc9.1_v08_00_00_30_run1_reco2_reco2 has 702 selected entries
file prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodx_mcc9.1_v08_00_00_30_run1_reco2_reco2 has 3185 selected entries
file prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodyz_mcc9.1_v08_00_00_31_run1_reco2_reco2 has 3714 selected entries
